In [ ]:
import os
import nibabel as nib
import pandas as pd
import numpy as np
from scipy.special import comb
import figure_utils
# import reload
from importlib import reload

project_id = 'K9'
project_dict = figure_utils.get_project_dict(project_id)[0]
num_clusters = 20

clusters, clusters_ID_list = figure_utils.get_clusters(
        project_dict, num_clusters
    )
for cluster_ID in clusters_ID_list:
    cluster = clusters[cluster_ID]
    print(f'Processing cluster {cluster_ID}...')

## Calculating homogeinity

In [ ]:
# Step 1. Calculate individuaL results into a single file for each cluster_ID
import os
import nibabel as nib
import pandas as pd
import numpy as np
from scipy.special import comb
import figure_utils
# import reload
from importlib import reload

reload(figure_utils)

cluster_number_list = [20,40,60,80,100,120,140,160,180,200,220,240,260,280,300]

project_dict = figure_utils.get_project_dict('K9')[0]
folder_to_save = r"C:\github\spatially_constrained_spectral_clustering\results"
# iterate over the number of clusters
for number_of_clusters in cluster_number_list:
    clusters,clusters_ID_list = figure_utils.get_clusters(project_dict, number_of_clusters)

    for cluster_ID in clusters_ID_list:
        # create file name
        file_name = folder_to_save + '\\' + 'homogeneity_' + str(number_of_clusters) + '_' + str(int(cluster_ID)).zfill(3) + '.csv'
        output_file = os.path.join(folder_to_save, file_name)
        
        # load results if exist
        if os.path.exists(file_name):
            # skip the file if it exists
            print('File already exists: ' + file_name)
            continue
            # results = pd.read_csv(file_name)
        else:
            results = pd.DataFrame(columns=['sub_N', 'average_corr', 'n_corr'])
            # save empty file
            results.to_csv(file_name, index=False)
        
        vox_list = clusters[cluster_ID]
        for sub_N in project_dict['Participants']:
            # print progress
            print('Processing subject ' + str(sub_N) + ' cluster ' + str(cluster_ID) + ' with ' + str(number_of_clusters) + ' clusters')
            # check if the subject is already in the results
            if sub_N in results['sub_N'].values:
                continue
            # get the average correlation and number of correlations
            average_corr, n_corr, datafolder = figure_utils.get_homogeinity(project_dict,sub_N,vox_list)
            row = pd.DataFrame([[sub_N, average_corr, n_corr]], columns=['sub_N', 'average_corr', 'n_corr'])
            results = pd.concat([results, row], ignore_index=True)

        # determine folder to save
        # folder_to_save = os.path.join(datafolder, project_dict['Dataset'], 'results', 'homogeneity')

        # create folder if it does not exist
        if not os.path.exists(folder_to_save):
            os.makedirs(folder_to_save)

        # save the results
        results.to_csv(output_file, index=False)
        print('Results saved in: ' + os.path.join(folder_to_save, file_name))

In [ ]:
# Step 2. Join individual results into a single file
cluster_number_list = [20,40,60,80,100,120,140,160,180,200,220,240,260,280,300]
database = 'K9'

project_dict = figure_utils.get_project_dict(database)[0]


experiment = 'CAPS'
folder_to_save = r"C:\github\spatially_constrained_spectral_clustering\results\cluster_homogeinity"
input_folder = r"C:\github\spatially_constrained_spectral_clustering\results\homogeneity"

# create an empty dataframe to store the results
data = pd.DataFrame()

for number_of_clusters in cluster_number_list:
    clusters,clusters_ID_list = figure_utils.get_clusters(project_dict, number_of_clusters)
    for cluster_ID in clusters_ID_list:
        # print progress
        print('Processing cluster ' + str(cluster_ID) + ' with ' + str(number_of_clusters) + ' clusters')
        # create file name
        file_name = input_folder + '\\' + 'homogeneity_' + str(number_of_clusters) + '_' + str(int(cluster_ID)).zfill(3) + '.csv'
        # load results
        results = pd.read_csv(file_name)
        print('Loaded results from: ' + file_name)
        # add column with number of clusters
        results['Number of clusters'] = number_of_clusters
        # add column with cluster ID
        results['Cluster ID'] = cluster_ID
        # add results to the dataframe
        data = pd.concat([data, results], ignore_index=True)

# save data to csv
data.to_csv(os.path.join(folder_to_save, 'homogeneity_' + database + '.csv'), index=False)
# print saved file
print('Results saved in: ' + os.path.join(folder_to_save, 'homogeneity_' + database + '.csv'))


In [ ]:
# Step 3. Generate slices for the Figure
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
import pandas as pd
import os
import copy  # Added import
from importlib import reload
import figure_utils
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable, get_cmap
#matplotlib.colormaps.get_cmap(obj)
# from matplotlib.colormaps import get_cmap
# import ListedColormap
from matplotlib.colors import ListedColormap

reload(figure_utils)

# Add options
zero_transparent = True  # If True, zeros in data_map and slice_base are set to np.nan
nan_transparent = True   # If True, NaNs in data_map are plotted as transparent; if False, print "decision pending"
alpha = 1.0               # Transparency of overlay
plot_type = 'm'  # 'm' for mean' or 'z' for Z-score
variable_name = 'Homogeneity'  # 'Homogeneity' 
write_file=True
clear_background=True
alphabetic_slides = True
min_val_assigned = [] # assign number to override the values in the colormap
max_val_assigned = [] # assign number to override the values in the colormap

specie = 'D'

results_folder = r"C:\github\spatially_constrained_spectral_clustering\results\cluster_homogeinity"
results_output_folder = r"C:\github\spatially_constrained_spectral_clustering\results\Figures"
results_file = os.path.join(results_folder, 'homogeneity_K9.csv')
# load the results
results = pd.read_csv(results_file)

# To calculate min and max values for the colormap
res_by_n_clusters = pd.DataFrame(columns=['number_of_clusters', 'average_corr', 'nan_count'])
# get number of clusters
cluster_possible_list = results['Number of clusters'].unique()
for N_clusters in cluster_possible_list:
    # Create a figure according to figure_dims
    res_df_sel = results[results['Number of clusters'] == N_clusters]
    new_row = {
        'number_of_clusters': N_clusters,
        'average_corr': np.nanmean(res_df_sel['average_corr'].values),
        'nan_count': np.sum(np.isnan(res_df_sel['average_corr'].values)),
    }
    # add new_row to res_by_n_clusters
    res_by_n_clusters = pd.concat([res_by_n_clusters, pd.DataFrame([new_row])], ignore_index=True)

# check if value is assigned to min_val_assigned
if min_val_assigned: # if min_val_assigned is not empty, set to the assigned value
    min_val = min_val_assigned
else: # if min_val_assigned is empty, calculate the minimum value
    min_val = res_by_n_clusters['average_corr'].min()
# check if value is assigned to max_val_assigned
if max_val_assigned: # if max_val_assigned is not empty, set to the assigned value
    max_val = max_val_assigned # 
else:
    max_val = res_by_n_clusters['average_corr'].max()

project_dict = figure_utils.get_project_dict('K9')[0]
figure_letter = 'B'
figures_folder = figure_utils.get_path('Figures', project_dict, figure_letter=figure_letter)

nii_folder = os.path.join(results_output_folder, "Figure_" + figure_letter)

img_type = "b_GreyMatter2mm"
base_img = "Czeibert_brain2mm"
atlas_type = project_dict['Atlas_type']
cmap_type = "hot" # takes hot 

alphabetic_slides=False

cluster_list = [20, 40, 60, 80, 100, 120, 140, 160, 180, 200, 220, 240, 260, 280, 300]

for number_of_clusters in cluster_list:
    # for number_of_clusters in cluster_list:
    segment_map_path = os.path.join(
        figure_utils.get_datafolder(username, local_data=True),
        project_dict['Dataset'],
        'hierarchical_clustering',
        f"{specie}-group-tcorr-{img_type}_{number_of_clusters}.nii.gz"
    )
    results_sel = results[results['Number of clusters'] == number_of_clusters]
    
    # load data_map
    data_map = nib.load(segment_map_path).get_fdata()
    # Convert data_map to float if necessary
    data_map = data_map.astype(float)
    
    cluster_ID_list = np.unique(data_map)
    # remove 0 from cluster_ID_list
    cluster_ID_list = cluster_ID_list[cluster_ID_list != 0]
    # Go through the cluster_ID_list
    for cluster_ID in cluster_ID_list:
        # filter results_sel for the cluster_ID
        results_sel_cluster = results_sel[results_sel['Cluster ID'] == cluster_ID]
        val = np.mean(results_sel_cluster['average_corr'])
        # if val is NaN, set it to 0
        if np.isnan(val):
            val = 0
            # print warning
            print(f"Warning: Cluster {cluster_ID} has NaN value, setting it to 0.")
        # assign val to each voxel in data_map that belongs to the cluster_ID
        data_map[data_map == cluster_ID] = val
    # get unique values in data_map
    unique_vals = np.unique(data_map)
    # check that data_map doesn't contain NaNs
    if np.isnan(data_map).any():
        # print("Warning: data_map contains NaNs")
        # trigger exception, stop program
        raise Exception("data_map contains NaNs. Please check the data.")

    # Define the colormap matplotlib.colormaps.get_cmap(obj)
    cmap = get_cmap(cmap_type)  # Choose your colormap, e.g., 'viridis', 'plasma', 'coolwarm', etc.
    norm = Normalize(vmin=min_val, vmax=max_val)

    # Create a scalar map to map values to colors
    scalar_map = ScalarMappable(norm=norm, cmap=cmap)

    # Map unique_vals to colors
    cmap_overlay = scalar_map.to_rgba(unique_vals)

    # add an initial row that represents alpha = 0
    cmap_overlay = np.vstack(([[0, 0, 0, 0]], cmap_overlay))
    cmap_overlay = ListedColormap(cmap_overlay)

    slice_dict = {
        'atlas_type': 'Nitzsche',
        'z': [32],
        'cmap': cmap_overlay,
        'specie': 'D',
    }

    prefix_out = 'Hom_' + str(number_of_clusters)


    figure_utils.plot_slice2(project_dict, prefix_out, figure_letter, slice_dict, data_map, alpha, alphabetic_slides, write_file, clear_background)

    # save data_map as nifti file
    nii = nib.Nifti1Image(data_map, affine=np.eye(4))
    # check if the folder exists, if not create it
    if not os.path.exists(nii_folder):
        os.makedirs(nii_folder)
        # print message
        print(f"Folder {nii_folder} created")
    # save the nii file
    nib.save(nii, os.path.join(nii_folder, f"{specie}_cluster_{number_of_clusters}.nii.gz"))
    # print message
    print(f"File saved in {os.path.join(nii_folder, f'{specie}_cluster_{number_of_clusters}.nii.gz')}")




In [ ]:
# Figure B - Homogeneity, by number of clusters

# Clear all the variables
%reset -f
import os
import nibabel as nib
import pandas as pd
import numpy as np
import figure_utils  # Ensure this is correctly imported or defined
from scipy.special import comb
from importlib import reload
from matplotlib import pyplot as plt
reload(figure_utils)
import matplotlib.ticker as mticker   # ← new import

figure_letter = 'B'
grid_spacing = 0.03

save_file = True
second_axis = False  # If True, add a second axis with the number of NaNs
color_comb = 'white' # tales 'white' or 'black'

# set custom limits

# set healthy to 0.40 and 0.70
custom_min_y = 0.05 # set number or NaN
custom_max_y = 0.25 # set number or NaN

if color_comb == 'white':
    opposite_color = 'black'
elif color_comb == 'black':
    opposite_color = 'white'
else:
    raise ValueError('color_comb must be white or black')

dataset = 'CAPS_K9'
experiment = 'CAPS'
figure_dims = (10, 1)
project_dict = figure_utils.get_project_dict('K9')[0]
figures_folder = figure_utils.get_path('Figures', project_dict, figure_letter=figure_letter)
# define path to results
res_path = "C:\github\spatially_constrained_spectral_clustering\results\cluster_homogeinity\homogeneity_K9.csv"
# setup path to save the table 'homogeneity_K9_per_N_clusters.csv'
results_file = r"C:\github\spatially_constrained_spectral_clustering\results\cluster_homogeinity\homogeneity_K9_per_N_clusters.csv"
res_df = pd.read_csv(res_path)

cluster_possible_list = res_df['Number of clusters'].unique()
res_by_n_clusters = pd.DataFrame()
for N_clusters in cluster_possible_list:
    # Create a figure according to figure_dims
    res_df_sel = res_df[res_df['Number of clusters'] == N_clusters]
    new_row = {
        'number_of_clusters': N_clusters,
        'average_corr': np.nanmean(res_df_sel['average_corr'].values),
        'nan_count': np.sum(np.isnan(res_df_sel['average_corr'].values)),
    }
    # add new_row to res_by_n_clusters
    res_by_n_clusters = pd.concat([res_by_n_clusters, pd.DataFrame([new_row])], ignore_index=True)
 

# save the results
res_by_n_clusters.to_csv(results_file, index=False)
print('Table saved to: ' + results_file)

# create figure
fig, ax = plt.subplots(figsize=figure_dims)
# get the y values and x values
y_values = res_by_n_clusters['average_corr'].values
x_values = res_by_n_clusters['number_of_clusters'].values
# plot the data
ax.plot(x_values, y_values, 'o-', label='Data', color=opposite_color)

# get min_y and max_y
min_y = min(y_values)
max_y = max(y_values)
# define percentage to add to the y-axis
perc = 0.1

# add 10% of the y-axis
if min_y < 0:
    min_y = min_y + min_y*perc
else:
    min_y = min_y - min_y*perc
if max_y < 0:
    max_y = max_y - max_y*perc
else:
    max_y = max_y + max_y*perc


# round min_y to the nearest 0.1
min_y = round(min_y - 0.01, 2)

# round max_y to the nearest 0.1
max_y = round(max_y + 0.01, 2)

# overwrite if limits are not NaN
if not np.isnan(custom_min_y):
    min_y = custom_min_y
if not np.isnan(custom_max_y):
    max_y = custom_max_y

ax.set_ylim([min_y, max_y])

# ─────────────── y-axis ticks & grid ───────────────
# major ticks: just min and max
ax.yaxis.set_major_locator(mticker.FixedLocator([min_y, max_y]))
# minor ticks: every grid_spacing
ax.yaxis.set_minor_locator(mticker.MultipleLocator(grid_spacing))
# draw grid only at minor ticks
ax.grid(which='minor', color='gray', linestyle='--', linewidth=0.5, alpha=0.5)
# optionally turn off any major grid
ax.grid(which='major', color='none')

# ensure only major tick labels appear
ax.tick_params(axis='y', which='minor', length=0)

# Set the background color to none
ax.set_facecolor('none')

for spine in ['top','left','right','bottom']:
    ax.spines[spine].set_visible(False)
# hide x-axis ticks and labels
ax.tick_params(axis='x', colors='none')
# color the y-axis labels
ax.tick_params(axis='y', colors=opposite_color)

if second_axis:
    # ------------------  second (right) axis – NEW scatter  ------------------
    ax2 = ax.twinx()                              # create secondary y-axis
    y_right = res_by_n_clusters['nan_count'].values
    ax2.scatter(x_values, y_right, color='red', zorder=3,
                label='NaN count', marker='o', s=40)

    # Optional: give the right axis its own limits & style
    perc = 0.1
    min_y2, max_y2 = y_right.min(), y_right.max()
    if min_y2 < 0:  min_y2 += min_y2*perc
    else:           min_y2 -= min_y2*perc
    if max_y2 < 0:  max_y2 -= max_y2*perc
    else:           max_y2 += max_y2*perc
    ax2.set_ylim(min_y2, max_y2)

    ax2.tick_params(axis='y', colors='red')       # make tick labels red
    ax2.spines['right'].set_visible(False)        # keep frameless look

# Show the plot
plt.show()

# Determine path
figure_path = os.path.join(figures_folder, 'Homogeneity_scatterplot' + '.png')
if save_file:
    # check that figures_folder exists, if not create it
    if not os.path.exists(figures_folder):
        os.makedirs(figures_folder)
        print('Folder created: ' + figures_folder)


    # Save the figure with transparency
    fig.savefig(figure_path, transparent=True)
    print('Figure saved to: ' + figure_path)
else:
    print(figure_path + ' not saved')
